# Importing Libraries

In [ ]:
!pip install ktrain

In [ ]:
import pandas as pd
import numpy as np

# Impartirea setului de date
from sklearn.model_selection import train_test_split

import ktrain
from ktrain import text

# Reading Dataset

In [ ]:
url = 'https://raw.githubusercontent.com/DanielaManate/SentimentAnalysis-TopicModeling/master/Data/Input/3.input_data_prepped_bow.csv'
reviews = pd.read_csv(url)
reviews.head(2)

# Train Test Split

In [ ]:
data_train, data_test, _, _ = train_test_split(
    reviews[['text', 'positive']],
    reviews['positive'],
    train_size=0.8,
    random_state=42)

In [ ]:
print(data_train.shape, data_test.shape)

In [ ]:
data_train.head(2)

# Pre-procesarea datelor cu ktrain
https://amaiya.github.io/ktrain/text/index.html

In [ ]:
(X_train, y_train), (X_test, y_test), preprocesor = text.texts_from_df(
    train_df=data_train,
    val_df=data_test,
    text_column='text',
    label_columns='positive',
    maxlen=128, #numarul maxim de cuvinte considerate dintr-o recenzie
    preprocess_mode='bert')

In [ ]:
X_train

In [ ]:
data_train['text'].iloc[0]

In [ ]:
print('Input IDs pentru a doua recenzie', X_train[0][1])

In [ ]:
print('Input IDs shape:', X_train[0].shape)

# Model Building

In [ ]:
bert = text.text_classifier(name='bert',
                            train_data=(X_train, y_train),
                            preproc=preprocesor)

Learning Rate = pasul de invatare

In [ ]:
learner = ktrain.get_learner(model=bert,
                             train_data=(X_train, y_train),
                             val_data=(X_test, y_test))
# learner.lr_find() # gasirea celui mai optim learning rate

In [ ]:
# 0.00003 - 3e-5

In [ ]:
learner.fit_onecycle(lr=3e-5, epochs=2)

In [ ]:
# Classification Report pe setul de test
learner.validate()

# Exemplu review care a fost clasificat gresit

In [ ]:
learner.view_top_losses(n=1, preproc=preprocesor)

In [ ]:
data_test['text'].iloc[894]

# Clasificarea altor recenzii

In [ ]:
pred = ktrain.get_predictor(learner.model,
                            preproc=preprocesor)

In [ ]:
data = ['The food was great',
        'This was the best pizza in town',
        "Worst pizza I've ever eaten",
        'Horrible service in this restaurant',
        "The service at this restaurant was ok"
        ]

In [ ]:
pred.predict(data)